<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Load-Settings" data-toc-modified-id="Load-Settings-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Load Settings</a></span></li><li><span><a href="#Load-Tables" data-toc-modified-id="Load-Tables-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Load Tables</a></span></li><li><span><a href="#Define-Functions" data-toc-modified-id="Define-Functions-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Define Functions</a></span></li></ul></div>

## Load Settings

In [1]:
import configparser
# ConfigParserオブジェクトを作成
config = configparser.ConfigParser()
# ファイルを読み込む
config.read('./config.ini', encoding='utf-8')

# セクションとオプションから値を取得
value_table_file = config.get('ForDebug', 'value_table_file')
#value_type = config.get('ForDebug', 'value_type')
value_range_ul_on_value_table = float( config.get('ForDebug', 'value_range_ul_on_value_table') )
value_range_ll_on_value_table = float( config.get('ForDebug', 'value_range_ll_on_value_table') )
if value_range_ul_on_value_table < value_range_ll_on_value_table: 
    tmp = value_range_ll_on_value_table
    value_range_ll_on_value_table = value_range_ul_on_value_table
    value_range_ul_on_value_table = tmp
#value_range_ul_on_vtk_file = float( config.get('ForDebug', 'value_range_ul_on_vtk_file') )
#value_range_ll_on_vtk_file = float( config.get('ForDebug', 'value_range_ll_on_vtk_file') )

print("value_table_file: " + value_table_file)
#print("value_type: " + value_type)
print("value_range_ul_on_value_table: " + str(value_range_ul_on_value_table))
print("value_range_ll_on_value_table: " + str(value_range_ll_on_value_table))
#print("value_range_ul_on_vtk_file: " + str(value_range_ul_on_vtk_file))
#print("value_range_ll_on_vtk_file: " + str(value_range_ll_on_vtk_file))

value_table_file: ./ValueTables/DKSurf_DivergingValues_TwoObject.tsv
value_range_ul_on_value_table: 1.0
value_range_ll_on_value_table: -1.0


## Load Tables

In [2]:
import pandas as pd

In [3]:
#
# Load "OBJECT_LABEL-VALUE" Tbale
#
#    OBJECT_LABEL: VtkfileTableと橋渡しするためのラベル
#    VALUE: 各オブジェクトに与えたい値
#

df_value_table = pd.read_csv(value_table_file, header=None, names=['OBJECT_LABEL', 'VALUE'], sep="\t")
print(df_value_table)

  OBJECT_LABEL     VALUE
0   L_bankssts  0.602898
1   R_bankssts  0.702898


In [4]:
#
# Load "OBJECT_LABEL-VTKFILE" Table
#
#     OBJECT_LABEL: ValueTableと橋渡しするためのラベル
#     VTK_FILE: ./vtk/without_val 下に存在するファイル
#
vtkfile_table_file = "./VtkfileTable.tsv"
df_file_table = pd.read_csv(vtkfile_table_file, header=0, names=['OBJECT_LABEL', 'VTK_FILE'], sep="\t")
print(df_file_table)

                 OBJECT_LABEL                                VTK_FILE
0                  L_bankssts                 001_ctx-lh-bankssts.vtk
1   L_caudalanteriorcingulate  002_ctx-lh-caudalanteriorcingulate.vtk
2       L_caudalmiddlefrontal      003_ctx-lh-caudalmiddlefrontal.vtk
3                    L_cuneus                   004_ctx-lh-cuneus.vtk
4                L_entorhinal               005_ctx-lh-entorhinal.vtk
..                        ...                                     ...
86                   R_accumb            088_Right-Accumbens-area.vtk
87                  L_LatVent                          Lt_LatVent.vtk
88                  R_LatVent                          Rt_LatVent.vtk
89                   L_Cortex                           Lt_ribbon.vtk
90                   R_Cortex                           Rt_ribbon.vtk

[91 rows x 2 columns]


## Define Functions

In [5]:
#
# Function for adding a value to the specified vtk file.
#
def infuseValueIntoAVtkFile(input_roi_vtk_file_path, output_destination, val_on_table, verbose):
    print("------------------------ addValueToAVtkFile() has been called ------------------------")
    
    print('input_roi_vtk_file_path: ' + input_roi_vtk_file_path)
    
    #
    # === Preparing output destinations ===
    #
    print('output_destination: ' + output_destination)
    
    import pathlib
    input_file_name = pathlib.Path(input_roi_vtk_file_path).name
    input_file_parent =  pathlib.Path(input_roi_vtk_file_path).parent
    ouput_file_path = "./" + str(pathlib.Path(output_destination)) + "/" +input_file_name
    output_file_parent = pathlib.Path(ouput_file_path).parent
    
    import os
    if not os.path.isfile(input_roi_vtk_file_path): 
        print("The input file does not exist. (do nothing)")
        return
    
    if output_file_parent == input_file_parent: 
        print("The parent of the specified output file is identical to the parent of the input file. Check arguments. (do nothing) ")
        return
    
    if verbose == True:
        print("input_file_name: " + input_file_name)
        print("ouput_file_path: " + ouput_file_path)
    
    output_folder=os.path.dirname(ouput_file_path)
    if  os.path.exists(output_folder): 
        print("    '" + output_folder + "' already exists.")
    else: 
        os.makedirs(output_folder, exist_ok=True)
        print("    '" + output_folder + "' was created.")
    

    # 
    # === Load A VTK File ===
    #
    input_file_path = input_roi_vtk_file_path

    f = open(input_file_path)
    lines = f.readlines() # 1行毎にファイル終端まで全て読む(改行文字も含まれる)
    f.close()
    
    # lines: リスト。要素は1行の文字列データ
    
    if verbose == True:
        for line in lines:
            print(line, end="")
        print()
    
    #
    # === Search the First Line of Data Field ===
    #
    start_line_num_of_data = 0
    for line_num in range(1, len(lines)): 
        if "NORMALS normals float\n" == lines[line_num]: 
            start_line_num_of_data = line_num + 1

    print( "The first line number of Data Field is "  + str(start_line_num_of_data) )

    if verbose == True:
        print("The first line of Data Filed: " + lines[start_line_num_of_data])
        print()
    
    #
    # === Add A Number to the Data Field ===
    #
    new_data_lines = [] 
    print("Now processing the line ", end="")
    infused_values = []
    for line_num in range( start_line_num_of_data, len(lines)): 
        print( str(line_num) + ". ", end="")
        line_editing = lines[line_num]
        line_editing = line_editing.rstrip('\n')
        values = line_editing.split(" ")
        #print(values)
        values.remove('')
        #print(values)
        values = [float(i) for i in values]
        #print(values)
        new_values = []
        
        #############################################
        # Sequential Values
        # e.g.: 0〜+100; 0〜+1; 0〜+X
        #
        # Diverging Values
        # e.g.: -1〜+1; -100〜+100; -X〜+X

        # Qualitative Values
        # e.g.: 1, 2, 3, 4, ...; 1, 3, 5, 7, ...; A, Y, Q, ...
        if value_range_ul_on_value_table > 0 and value_range_ll_on_value_table < 0:
            value_type = "DIVERGING"
        else:
            value_type = "SEQUENTIAL"
        
        if value_type == "DIVERGING": 
            base_range_on_value_table = value_range_ul_on_value_table - 0
            fluctuation_factor = base_range_on_value_table / 5
            for value in values: 
                new_value =  val_on_table + value * fluctuation_factor
                new_values.append( new_value )
            #base_range_on_value_table = (value_range_ul_on_value_table - 0)
            #base_range_on_vtk_file = (value_range_ul_on_vtk_file - 0)
            #fluctuation_factor = base_range_on_vtk_file / 5
            #for value in values: 
            #    new_value =  ( (val_on_table/base_range_on_value_table) * base_range_on_vtk_file )  + 0 + ( value * fluctuation_factor)
            #    new_values.append( new_value )
        elif value_type == "SEQUENTIAL": 
            base_range_on_value_table = (value_range_ul_on_value_table - value_range_ll_on_value_table)
            fluctuation_factor = base_range_on_value_table / 5
            for value in values: 
                new_value =  val_on_table  + ( value * fluctuation_factor)
                new_values.append( new_value )
            #base_range_on_value_table = (value_range_ul_on_value_table - value_range_ll_on_value_table)
            #base_range_on_vtk_file = value_range_ul_on_vtk_file - value_range_ll_on_vtk_file
            #fluctuation_factor = base_range_on_vtk_file / 5
            #for value in values: 
            #    new_value =  ( (val_on_table/base_range_on_value_table) * base_range_on_vtk_file ) + value_range_ll_on_vtk_file  + ( value * fluctuation_factor)
            #    new_values.append( new_value )
        elif value_type == "QUALITAIVE": 
            print("Error. Not yet coded.")
            import sys; sys.exit()
        else: 
            print("Error. Please check 'value_type'.")
            import sys; sys.exit()
        
        #
        # For Infused Value Check
        #
        #print(new_values)
        #if len(new_values) > 1:
        #    import statistics
        #    print(str(statistics.mean(new_values)) + "±" + (str(statistics.stdev(new_values))) )
        for new_value in new_values: 
            infused_values.append(new_value)
            
        #############################################
        
        new_values_str = [str(i) for i in new_values]
        #new_values_str = new_values_str.append(" ")
        #new_values_str = new_values_str.append("\n")
        new_values_str = ' '.join(new_values_str)
        new_values_str = new_values_str + " \n"
        #print(new_values_str)
        new_data_lines.append(new_values_str)
        

        
    print("...done.")
    
    if verbose == True:
        print("")
        print("new_data_lines: ")
        print(new_data_lines)
        print("")
    
    print()
    
    #
    # === Output As A New File ===
    #
    print("Start output...")
    new_lines = []
    new_lines.append('# vtk DataFile Version 4.0\n')
    for line_num in range(1, len(lines)): 
        #print(str(line_num) + ": ", end="")
        if line_num < start_line_num_of_data: 
            #print(lines[line_num])
            new_lines.append(lines[line_num])
        elif line_num >= start_line_num_of_data: 
            j = line_num - start_line_num_of_data
            #print(new_data_lines[j])
            new_lines.append(new_data_lines[j])
    print("")
    
    if verbose == True:
        print("new_lines: ")
        print(new_lines)
        print()
    
    
    with open(ouput_file_path, mode='w') as f:
        f.writelines(new_lines)
    
    
    import statistics
    infused_value_stats = str(statistics.mean(infused_values)) + "±" + (str(statistics.stdev(infused_values)))
    
    
    print("'" + ouput_file_path + "' was generated. (Infused Value: " + infused_value_stats + ")" )
    print("--------------------------------------------------------------------------------------")
    print()
    
            
#infuseValueIntoAVtkFile(
#    input_roi_vtk_file_path="./vtk/Lt_Brain_NonCortical/074_Left-Thalamus.vtk", 
#    output_destination="./vtk/Lt_Brain_NonCortical_with_Val", 
#    val_on_table=10, verbose=True)

In [6]:
#df_value_file_table = pd.merge(df_file_table, df_value_table, on="OBJECT_LABEL", how="left") #file_tableにあるもの
df_value_file_table = pd.merge(df_file_table, df_value_table, on="OBJECT_LABEL", how="right") #value_tableにあるもの
#df_value_file_table = pd.merge(df_file_table, df_value_table, on="OBJECT_LABEL", how="outer") #両方にあるものを把握
df_value_file_table

,OBJECT_LABEL,VTK_FILE,VALUE
0,L_bankssts,001_ctx-lh-bankssts.vtk,0.602898
1,R_bankssts,035_ctx-rh-bankssts.vtk,0.702898


In [7]:
#
# Infuse Values into Vtk Files
# 
src_vtk_folder = "./vtk/without_val"
dest_vtk_folder = "./vtk/with_val"

import os
if os.path.isdir(dest_vtk_folder): 
    import shutil
    shutil.rmtree(dest_vtk_folder)
    os.mkdir(dest_vtk_folder)

for r in range(0, len(df_value_file_table)):
    #print(r)
    dest_vtk_file = df_value_file_table["VTK_FILE"][r]
    obj_val = df_value_file_table["VALUE"][r]
    print("Infuse " + str(obj_val) + " into " + dest_vtk_file)
    infuseValueIntoAVtkFile(
        input_roi_vtk_file_path=src_vtk_folder + "/" + dest_vtk_file, 
        output_destination=dest_vtk_folder, 
        val_on_table=obj_val, verbose=False)

Infuse 0.602897737640887 into 001_ctx-lh-bankssts.vtk
------------------------ addValueToAVtkFile() has been called ------------------------
input_roi_vtk_file_path: ./vtk/without_val/001_ctx-lh-bankssts.vtk
output_destination: ./vtk/with_val
    './vtk/with_val' already exists.
The first line number of Data Field is 1761
Now processing the line 1761. 1762. 1763. 1764. 1765. 1766. 1767. 1768. 1769. 1770. 1771. 1772. 1773. 1774. 1775. 1776. 1777. 1778. 1779. 1780. 1781. 1782. 1783. 1784. 1785. 1786. 1787. 1788. 1789. 1790. 1791. 1792. 1793. 1794. 1795. 1796. 1797. 1798. 1799. 1800. 1801. 1802. 1803. 1804. 1805. 1806. 1807. 1808. 1809. 1810. 1811. 1812. 1813. 1814. 1815. 1816. 1817. 1818. 1819. 1820. 1821. 1822. 1823. 1824. 1825. 1826. 1827. 1828. 1829. 1830. 1831. 1832. 1833. 1834. 1835. 1836. 1837. 1838. 1839. 1840. 1841. 1842. 1843. 1844. 1845. 1846. 1847. 1848. 1849. 1850. 1851. 1852. 1853. 1854. 1855. 1856. 1857. 1858. 1859. 1860. 1861. 1862. 1863. 1864. 1865. 1866. 1867. 1868. 1869

Now processing the line 1604. 1605. 1606. 1607. 1608. 1609. 1610. 1611. 1612. 1613. 1614. 1615. 1616. 1617. 1618. 1619. 1620. 1621. 1622. 1623. 1624. 1625. 1626. 1627. 1628. 1629. 1630. 1631. 1632. 1633. 1634. 1635. 1636. 1637. 1638. 1639. 1640. 1641. 1642. 1643. 1644. 1645. 1646. 1647. 1648. 1649. 1650. 1651. 1652. 1653. 1654. 1655. 1656. 1657. 1658. 1659. 1660. 1661. 1662. 1663. 1664. 1665. 1666. 1667. 1668. 1669. 1670. 1671. 1672. 1673. 1674. 1675. 1676. 1677. 1678. 1679. 1680. 1681. 1682. 1683. 1684. 1685. 1686. 1687. 1688. 1689. 1690. 1691. 1692. 1693. 1694. 1695. 1696. 1697. 1698. 1699. 1700. 1701. 1702. 1703. 1704. 1705. 1706. 1707. 1708. 1709. 1710. 1711. 1712. 1713. 1714. 1715. 1716. 1717. 1718. 1719. 1720. 1721. 1722. 1723. 1724. 1725. 1726. 1727. 1728. 1729. 1730. 1731. 1732. 1733. 1734. 1735. 1736. 1737. 1738. 1739. 1740. 1741. 1742. 1743. 1744. 1745. 1746. 1747. 1748. 1749. 1750. 1751. 1752. 1753. 1754. 1755. 1756. 1757. 1758. 1759. 1760. 1761. 1762. 1763. 1764. 1765. 1766